In [1]:
import os
import warnings
import numpy as np
import pandas as pd
#import functions

from lambeq import BobcatParser

warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

ModuleNotFoundError: No module named 'lambeq'

In [2]:
twitter_data_pos = pd.read_csv("processedPositive.csv")
twitter_data_neu = pd.read_csv("processedNeutral.csv")
twitter_data_neg = pd.read_csv("processedNegative.csv")

all_data = pd.DataFrame()

pos_tweets = twitter_data_pos.columns
num_pos = pos_tweets.size
neu_tweets = twitter_data_neu.columns
num_neu = neu_tweets.size
neg_tweets = twitter_data_neu.columns
num_neg = neg_tweets.size

all_tweets = pos_tweets.append(neu_tweets).append(neg_tweets)
values = [1 for i in range(num_pos)] + [0 for i in range(num_neu)] + [-1 for i in range(num_neg)]

all_data["Tweets"] = all_tweets
all_data["Score"] = values
all_data.head()

,Tweets,Score
0,An inspiration in all aspects: Fashion,1
1,fitness,1
2,beauty and personality. :)KISSES TheFashionIcon,1
3,Apka Apna Awam Ka Channel Frankline Tv Aam Adm...,1
4,Beautiful album from the greatest unsung guit...,1


In [3]:
len(all_tweets)

4326

In [4]:
shuffled = all_data.sample(frac=1)
shuffled.head()

,Tweets,Score
3082,Why African envoys are upset,-1
1369,to look beyond defence during visit.,0
174,Wow i just made a JS compressor that actually ...,1
2238,and cowardice; season of buds and thorns,0
2486,By noon,0


In [9]:
def less_than_8(string):
    if len(string.split()) < 8:
        return True
    else:
        return False
less_than_8("I am going to the park today and eating food")

False

In [6]:
# quantum time
parser = BobcatParser()

In [15]:
indices = shuffled["Tweets"].apply(less_than_8)
new_shuffled = shuffled.loc[indices]
new_shuffled

,Tweets,Score
3082,Why African envoys are upset,-1
1369,to look beyond defence during visit.,0
2238,and cowardice; season of buds and thorns,0
2486,By noon,0
1069,Slayin_Bodies Going Live...Please Tune In...Th...,1
...,...,...
3438,Calcutta HC sends case to CBI.,-1
2377,and more. Also in epaper. .39,0
2410,a day ahead of confidence vote.,0
1588,India rushes to put out fire.,0


In [17]:
train_cutoff = (0.8 * len(new_shuffled["Tweets"])) // 1
twitter_training = new_shuffled.loc[:train_cutoff].copy()
twitter_testing = new_shuffled.loc[train_cutoff:].copy()

train, trainlabels = twitter_training.iloc[:,0].to_numpy(), twitter_training.iloc[:,1].to_numpy()
test, testlabels = twitter_testing.iloc[:,0].to_numpy(), twitter_testing.iloc[:,1].to_numpy()

In [26]:
train_sentences = list(train)
train_sentences[:5]

['Why African envoys are upset',
 ' to look beyond defence during visit. ',
 ' and cowardice; season of buds and thorns',
 'By noon',
 'Slayin_Bodies Going Live...Please Tune In...Thank You happy']

In [27]:
train_diagrams_0 = parser.sentences2diagrams(train_sentences[:5])

Tagging sentences:   0%|          | 0/2 [00:00<?, ?it/s]

Parsing tagged sentences:   0%|          | 0/5 [00:00<?, ?it/s]

Parse trees to diagrams:   0%|          | 0/5 [00:00<?, ?it/s]

In [28]:
from lambeq import AtomicType, IQPAnsatz, remove_cups
N = AtomicType.NOUN
S = AtomicType.SENTENCE
P = AtomicType.PREPOSITIONAL_PHRASE
PUNC = AtomicType.PUNCTUATION
C = AtomicType.CONJUNCTION

In [29]:
ansatz = IQPAnsatz({N: 1,S: 0, P: 1, PUNC: 0, C: 0},
                   n_layers=2, n_single_qubit_params=2)
try:
    train_circuits = [ansatz(remove_cups(diagram)) for diagram in train_diagrams_0]
except ValueError:
    print(diagram)